In [14]:
import pandas as pd 
import warnings
import numpy as np
import pickle
from pathlib import Path
import os 

warnings.filterwarnings("ignore")

# ============================================================================
# 1. 데이터 로드 및 전처리
# ============================================================================
print("="*80)
print("데이터 로드 및 전처리")
print("="*80)


# train_df와 test_df 로드
train_df = pd.read_csv("/workspace/data/tune_data/tune_df.csv")
test_df = pd.read_csv("/workspace/data/tune_data/test_df.csv")

print(f"Train 데이터 크기: {len(train_df)}")
print(f"Test 데이터 크기: {len(test_df)}")

# y = 타겟 변수 / x = 예측 변수
except_cols = ['Z_KDBDRS', 'K_ODD', 'KDBDRS', 'wt_s', 'DICCD_CP', 'ODD_Cur', 'ODD_Pas', 'DICCD_C', 'DICCD_P', 'MentD']
standard_cols = ['ZAI_Incom', 'Z_K_ODD', 'Z_K_CD', 'Z_K_IA', 'Z_K_HI', 'Z_GAD', 'Z_PHQ', 'Z_SAS']
nonstandard_cols = ['Incom', 'K_ODD', 'K_CD', 'K_IA', 'K_HI', 'GAD', 'PHQ', 'SAS']

drop_cols = except_cols + nonstandard_cols

# Train 데이터 전처리
print(f"\nTrain 데이터 - 기존 변수 수: {len(train_df.columns)}")
train_df = train_df.drop(columns=[col for col in drop_cols if col in train_df.columns])
print(f"Train 데이터 - 남은 변수 수: {len(train_df.columns)}")

# Test 데이터 전처리
print(f"\nTest 데이터 - 기존 변수 수: {len(test_df.columns)}")
test_df = test_df.drop(columns=[col for col in drop_cols if col in test_df.columns])
print(f"Test 데이터 - 남은 변수 수: {len(test_df.columns)}")

from utils.data_imputation import filter_by_missing_ratio

# Train 데이터 결측치 처리
train_df = filter_by_missing_ratio(train_df, threshold=0.25, visualize=False)
print(f"Train 데이터 결측치 처리 후 크기: {len(train_df)}")

# Test 데이터 결측치 처리
test_df = filter_by_missing_ratio(test_df, threshold=0.25, visualize=False)
print(f"Test 데이터 결측치 처리 후 크기: {len(test_df)}")


데이터 로드 및 전처리
Train 데이터 크기: 3262
Test 데이터 크기: 120

Train 데이터 - 기존 변수 수: 60
Train 데이터 - 남은 변수 수: 43

Test 데이터 - 기존 변수 수: 60
Test 데이터 - 남은 변수 수: 43
삭제 칼럼들 : ['GEdu', 'GAlc', 'GTob']

==== 결측치가 25% 미만인 컬럼만 남김 ====
남은 변수 수: 40
남은 컬럼 리스트: ['Sex', 'Answ', 'Z_K_IA', 'Z_K_HI', 'Z_K_ODD', 'Z_K_CD', 'ODD_CP', 'Age_Grp', 'MEdu', 'FEdu', 'P_Marr', 'MJob', 'FJob', 'ZAI_Incom', 'MAlc', 'FAlc', 'MTob', 'FTob', 'PSleep', 'Z_GAD', 'Z_PHQ', 'PAF', 'SBV', 'SBP', 'CBV', 'CBP', 'Avg_G', 'GDec', 'BF', 'RFG', 'AdolSlp', 'MoodD', 'AnxD', 'ST1', 'ST2', 'ST3', 'ST4', 'IGD_P', 'Z_SAS', 'SRD_CP']

==== 남아있는 컬럼 중 결측치가 있는 컬럼의 결측치 개수 ====
MEdu      56
FEdu     200
MJob      56
FJob     200
MAlc      56
FAlc     200
MTob      56
FTob     200
IGD_P    715
dtype: int64

결측치가 남아있는 변수 수: 9 / 전체 변수 수: 40
Train 데이터 결측치 처리 후 크기: 3262
삭제 칼럼들 : ['GEdu', 'GAlc', 'GTob']

==== 결측치가 25% 미만인 컬럼만 남김 ====
남은 변수 수: 40
남은 컬럼 리스트: ['Sex', 'Answ', 'Z_K_IA', 'Z_K_HI', 'Z_K_ODD', 'Z_K_CD', 'ODD_CP', 'Age_Grp', 'MEdu', 'FEdu', 'P_Marr', 'M

In [17]:
test_df['Sex'].value_counts()

Sex
1    67
2    53
Name: count, dtype: int64

In [13]:

from utils.data_preprocessor import check_preprocessing_needs, preprocess_dataframe
from utils.data_preprocessor import data_preprocess_pipeline

# Train 데이터 전처리
X_train_raw = train_df.drop(columns=['ODD_CP'])
y_train_raw = train_df['ODD_CP']
print(X_train_raw['Sex'].value_counts())
X_train_raw = preprocess_dataframe(
    X_train_raw, 
    target_col='ODD_CP',
    drop_weight=True,
    convert_categorical=['Answ', 'IGD_P', 'FEdu', 'MEdu', 'FJob', 'MJob', 'Age_Grp', 'P_Marr'],
    convert_ordinal=['ST1', 'ST2', 'ST3', 'ST4', 'PAF', 'MAlc', 'FAlc', "MTob", "FTob", "MAlc", "FAlc", "GAlc", "MTob", "FTob", "GTob"], 
    convert_binary=['Sex','SRD_CP', 'IGD_P', 'PSleep', 'SBV', 'SBP', 'CBV', 'CBP', 'GDec', 'BF', 'RFG', 'MentD', 'AdolSlp', 'MoodD', 'AnxD'],
    drop_low_variance=False,
    drop_leakage=True
)
print(X_train_raw['Sex'].value_counts())

X_train_raw = data_preprocess_pipeline(X_train_raw)
print(X_train_raw['Sex'].value_counts())

# Test 데이터 전처리 (동일한 전처리 적용)
X_test_raw = test_df.drop(columns=['ODD_CP'])
y_test_raw = test_df['ODD_CP']

X_test_raw = preprocess_dataframe(
    X_test_raw, 
    target_col='ODD_CP',
    drop_weight=True,
    convert_categorical=['Answ', 'IGD_P', 'FEdu', 'MEdu', 'FJob', 'MJob', 'Age_Grp', 'P_Marr'],
    convert_ordinal=['ST1', 'ST2', 'ST3', 'ST4', 'PAF', 'MAlc', 'FAlc', "MTob", "FTob", "MAlc", "FAlc", "GAlc", "MTob", "FTob", "GTob"], 
    convert_binary=['Sex', 'SRD_CP', 'IGD_P', 'PSleep', 'SBV', 'SBP', 'CBV', 'CBP', 'GDec', 'BF', 'RFG', 'MentD', 'AdolSlp', 'MoodD', 'AnxD'],
    drop_low_variance=False,
    drop_leakage=True
)
X_test_raw = data_preprocess_pipeline(X_test_raw)

# 컬럼 일치 확인 및 정렬
common_cols = list(set(X_train_raw.columns) & set(X_test_raw.columns))
X_train_raw = X_train_raw[common_cols]
X_test_raw = X_test_raw[common_cols]


Sex
2    1658
1    1604
Name: count, dtype: int64
Sex
1    1658
0    1604
Name: count, dtype: int64
▶ integer 전처리 중...
▶ float 전처리 중...
▶ 순서 있는 category(ordinal) 전처리 중...
▶ object 전처리 중...
▶ 순서 없는 category 전처리 중...
✅ 데이터 전처리 완료
Sex
 0.983581    1658
-1.016694    1604
Name: count, dtype: int64
▶ integer 전처리 중...
▶ float 전처리 중...
▶ 순서 있는 category(ordinal) 전처리 중...
▶ object 전처리 중...
▶ 순서 없는 category 전처리 중...
✅ 데이터 전처리 완료


In [11]:
X_test_raw['Sex'].value_counts()

Sex
-0.889407    67
 1.124345    53
Name: count, dtype: int64

In [ ]:
from utils.data_splitter import oversample_train_test_split, downsample_train_test_split
from utils.data_preprocessor import extract_features_with_onehot
from utils.ml_model import MultiModelFoldTrainer

# 샘플링 방법 정의 (Downsample + SMOTEEN만 사용)
oversample_method = "SMOTEEN"  # SMOTE / SMOTEEN / ADASYN / SMOTETomek
sampling_name = 'downsample_SMOTEEN'
sampling_config = {
    'type': 'oversample',
    'params': {'train_size_per_class': 240, 'method': 'SMOTEENN'}
}

# 실험 설정 테이블
EXPERIMENTS = [
    # Feature Set, Gender, Feature File, Exp Name
    ('A', 'A', '/workspace/data/results/top7_total.csv', '4_EXP_Methods_A_A'),      # 부모+청소년 / 전체
    ('A', 'M', '/workspace/data/results/top7_total.csv', '4_EXP_Methods_A_M'),      # 부모+청소년 / 남
    ('A', 'FM', '/workspace/data/results/top7_total.csv', '4_EXP_Methods_A_FM'),  # 부모+청소년 / 여
    ('P', 'A', '/workspace/data/results/top7_parent.csv', '4_EXP_Methods_P_A'),     # 부모 / 전체
    ('P', 'M', '/workspace/data/results/top7_parent.csv', '4_EXP_Methods_P_M'),     # 부모 / 남
    ('P', 'FM', '/workspace/data/results/top7_parent.csv', '4_EXP_Methods_P_FM'),  # 부모 / 여
    ('Ad', 'A', '/workspace/data/results/top7_adolescent.csv', '4_EXP_Methods_Ad_A'),   # 청소년 / 전체
    ('Ad', 'M', '/workspace/data/results/top7_adolescent.csv', '4_EXP_Methods_Ad_M'),   # 청소년 / 남
    ('Ad', 'FM', '/workspace/data/results/top7_adolescent.csv', '4_EXP_Methods_Ad_FM'), # 청소년 / 여
]

# 결과 저장 디렉토리 생성
results_dir = Path("/workspace/data/results")
results_dir.mkdir(parents=True, exist_ok=True)